### File to compile results at both intraoperative and preoperative level for all outcomes

#### Input: pred_stage, outcome name, result file location 

In [1]:
import pickle
import json
import os
import numpy as np
import pandas as pd
import math
from datetime import datetime
import matplotlib.pyplot as plt
import pprint

In [19]:
# result location 
pred_stage = 'Preoperative'  # {'Preoperative', 'Intraoperative'}
outcome_list= ['icu', 'mortality', 'aki2']
reg_outcome_list =['postop_los', 'opioids_count_day0', 'opioids_count_day1']
sav_dir = '/mnt/ris/sandhyat/Output-TS_docker_July2024/Best_results/' + pred_stage +"/"
model_name_list = ['XGBT', 'Scarf', 'TabNet']
# sav_dir = '/home/trips/PeriOperative_RiskPrediction/Best_results/' + pred_stage +"/"

In [27]:
comb_flag =0
for outcome in outcome_list:
    perf_filename = sav_dir + str(outcome) + '_Best_perf_metrics_combined_' + pred_stage.lower() + '.pickle'
    with open(perf_filename, 'rb') as file:
        metric_data = pickle.load(file)
#     model_name_list = list(metric_data.keys())
    ablation_name_list = list(metric_data[model_name_list[0]].keys())
    ablation_name_list_pretty = [i.split("_",1)[1] for i in ablation_name_list]
    
    df_auroc = pd.DataFrame(columns=model_name_list, index=ablation_name_list)
    df_auprc = pd.DataFrame(columns=model_name_list, index=ablation_name_list)

    for key, value in metric_data.items():
        for ab in ablation_name_list:
            df_auroc.loc[ab, key] = np.round(np.mean(value[ab][:,0]), decimals=3)
            df_auprc.loc[ab, key] = np.round(np.mean(value[ab][:,1]), decimals=3)

    # renaming the index
    df_auroc = df_auroc.rename(index=dict(zip(df_auroc.index, ablation_name_list_pretty)))
    df_auprc = df_auprc.rename(index=dict(zip(df_auprc.index, ablation_name_list_pretty)))
    
    new_columns = [(outcome, col) for col in df_auroc.columns]
    new_column_index = pd.MultiIndex.from_tuples(new_columns)
    
    df_auroc.columns = new_column_index
    df_auprc.columns = new_column_index

    if comb_flag==0:
        AUROC_comb = df_auroc
        AUPRC_comb = df_auprc
    else:
        AUROC_comb = pd.concat([AUROC_comb, df_auroc], axis=1)
        AUPRC_comb = pd.concat([AUPRC_comb, df_auprc], axis=1)
    comb_flag = comb_flag + 1
#     print(" Outcome ", outcome, " AUROC")
#     pprint.pp(df_auroc)
#     print("\n Outcome ", outcome, " AUPRC")
#     pprint.pp(df_auprc)
# pprint.pp(AUROC_comb)
# pprint.pp(AUPRC_comb)
print(AUROC_comb)
auroc_latex = AUROC_comb.to_latex()
print(auroc_latex)

                                     icu               mortality         \
                                    XGBT  Scarf TabNet      XGBT  Scarf   
preops_cbow                        0.969  0.909  0.954     0.943  0.897   
preops_cbow_homemeds               0.974  0.824   0.95     0.947   0.89   
preops_cbow_pmh_problist_homemeds  0.974  0.877  0.948     0.945  0.873   

                                           aki2                
                                  TabNet   XGBT  Scarf TabNet  
preops_cbow                        0.909  0.948  0.849   0.91  
preops_cbow_homemeds               0.911   0.95  0.856   0.91  
preops_cbow_pmh_problist_homemeds  0.916  0.952  0.836  0.906  
\begin{tabular}{llllllllll}
\toprule
{} & \multicolumn{3}{l}{icu} & \multicolumn{3}{l}{mortality} & \multicolumn{3}{l}{aki2} \\
{} &   XGBT &  Scarf & TabNet &      XGBT &  Scarf & TabNet &   XGBT &  Scarf & TabNet \\
\midrule
preops\_cbow                       &  0.969 &  0.909 &  0.954 &     0.943 &  

/tmp/ipykernel_2482294/1720213116.py:42: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  auroc_latex = AUROC_comb.to_latex()


In [21]:
comb_flag=0
for outcome in reg_outcome_list:
    perf_filename = sav_dir + str(outcome) + '_Best_perf_metrics_combined_' + pred_stage.lower() + '.pickle'
    with open(perf_filename, 'rb') as file:
        metric_data = pickle.load(file)
#     model_name_list = list(metric_data.keys())
    ablation_name_list = list(metric_data[model_name_list[0]].keys())
    ablation_name_list_pretty = [i.split("_",1)[1] for i in ablation_name_list]
    
    df_Corr = pd.DataFrame(columns=model_name_list, index=ablation_name_list)
    df_R2 = pd.DataFrame(columns=model_name_list, index=ablation_name_list)
    
    for key, value in metric_data.items():
        for ab in ablation_name_list:
            try:
                df_Corr.loc[ab, key] = np.round(np.mean(value[ab][:,0]), decimals=3)
                df_R2.loc[ab, key] = np.round(np.mean(value[ab][:,2]), decimals=3)
            except(KeyError):
                df_Corr.loc[ab, key] = 'TBF'
                df_R2.loc[ab, key] = 'TBF'

    # renaming the index
    df_Corr = df_Corr.rename(index=dict(zip(df_Corr.index, ablation_name_list_pretty)))
    df_R2 = df_R2.rename(index=dict(zip(df_R2.index, ablation_name_list_pretty)))
    
    new_columns = [(outcome, col) for col in df_Corr.columns]
    new_column_index = pd.MultiIndex.from_tuples(new_columns)
    
    df_Corr.columns = new_column_index
    df_R2.columns = new_column_index
    if comb_flag==0:
        Corr_comb = df_Corr
        R2_comb = df_R2
    else:
        Corr_comb = pd.concat([Corr_comb, df_Corr], axis=1)
        R2_comb = pd.concat([R2_comb, df_R2], axis=1)
    comb_flag = comb_flag + 1

#     print(" Outcome ", outcome, " Correlation")
#     pprint.pp(df_Corr)
#     print("\n Outcome ", outcome, " R2 score")
#     pprint.pp(df_R2)
pprint.pp(R2_comb)
R2_latex = R2_comb.to_latex()
print(R2_latex)

                                  postop_los               opioids_count_day0  \
                                        XGBT  Scarf TabNet               XGBT   
preops_cbow                            0.437  0.235  0.383              0.315   
preops_cbow_homemeds                   0.461  0.224  0.213              0.319   
preops_cbow_pmh_problist_homemeds      0.575  0.187    0.5              0.317   

                                                opioids_count_day1         \
                                   Scarf TabNet               XGBT  Scarf   
preops_cbow                        0.108  0.257              0.425  0.234   
preops_cbow_homemeds               0.087  0.264              0.431  0.192   
preops_cbow_pmh_problist_homemeds  0.074  0.192              0.425  0.216   

                                          
                                  TabNet  
preops_cbow                          0.4  
preops_cbow_homemeds               0.352  
preops_cbow_pmh_problist_homemeds  0

/tmp/ipykernel_2482294/1671556659.py:44: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  R2_latex = R2_comb.to_latex()


In [26]:
comb_flag =0
for outcome in outcome_list:
    perf_filename = sav_dir + str(outcome) + '_Best_perf_metrics_combined_' + pred_stage.lower() + 'Wave2.pickle'
    with open(perf_filename, 'rb') as file:
        metric_data = pickle.load(file)
#     model_name_list = list(metric_data.keys())
    ablation_name_list = list(metric_data[model_name_list[0]].keys())
    ablation_name_list_pretty = [i.split("_",1)[1] for i in ablation_name_list]
    
    df_auroc = pd.DataFrame(columns=model_name_list, index=ablation_name_list)
    df_auprc = pd.DataFrame(columns=model_name_list, index=ablation_name_list)

    for key, value in metric_data.items():
        for ab in ablation_name_list:
            try:
                df_auroc.loc[ab, key] = np.round(np.mean(value[ab][:,0]), decimals=3)
                df_auprc.loc[ab, key] = np.round(np.mean(value[ab][:,1]), decimals=3)
            except(KeyError):
                df_auroc.loc[ab, key] = 'TBF'
                df_auprc.loc[ab, key] = 'TBF'

    # renaming the index
    df_auroc = df_auroc.rename(index=dict(zip(df_auroc.index, ablation_name_list_pretty)))
    df_auprc = df_auprc.rename(index=dict(zip(df_auprc.index, ablation_name_list_pretty)))
    
    new_columns = [(outcome, col) for col in df_auroc.columns]
    new_column_index = pd.MultiIndex.from_tuples(new_columns)
    
    df_auroc.columns = new_column_index
    df_auprc.columns = new_column_index

    if comb_flag==0:
        AUROC_comb = df_auroc
        AUPRC_comb = df_auprc
    else:
        AUROC_comb = pd.concat([AUROC_comb, df_auroc], axis=1)
        AUPRC_comb = pd.concat([AUPRC_comb, df_auprc], axis=1)
    comb_flag = comb_flag + 1
#     print(" Outcome ", outcome, " AUROC")
#     pprint.pp(df_auroc)
#     print("\n Outcome ", outcome, " AUPRC")
#     pprint.pp(df_auprc)
# pprint.pp(AUROC_comb)
# pprint.pp(AUPRC_comb)
print(AUROC_comb)
auroc_latex = AUROC_comb.to_latex()
print(auroc_latex)

                                     icu               mortality         \
                                    XGBT  Scarf TabNet      XGBT  Scarf   
preops_cbow                        0.847   0.61  0.952     0.885  0.785   
preops_cbow_homemeds               0.865   0.65  0.928     0.885   0.79   
preops_cbow_pmh_problist_homemeds  0.859  0.638  0.922     0.859  0.708   

                                           aki2                
                                  TabNet   XGBT  Scarf TabNet  
preops_cbow                        0.757  0.916  0.777   0.84  
preops_cbow_homemeds               0.767  0.917    0.8   0.84  
preops_cbow_pmh_problist_homemeds  0.709  0.917  0.747  0.854  
\begin{tabular}{llllllllll}
\toprule
{} & \multicolumn{3}{l}{icu} & \multicolumn{3}{l}{mortality} & \multicolumn{3}{l}{aki2} \\
{} &   XGBT &  Scarf & TabNet &      XGBT &  Scarf & TabNet &   XGBT &  Scarf & TabNet \\
\midrule
preops\_cbow                       &  0.847 &   0.61 &  0.952 &     0.885 &  

/tmp/ipykernel_2482294/873384497.py:46: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  auroc_latex = AUROC_comb.to_latex()


In [25]:
comb_flag=0
for outcome in reg_outcome_list:
    perf_filename = sav_dir + str(outcome) + '_Best_perf_metrics_combined_' + pred_stage.lower() + 'Wave2.pickle'
    with open(perf_filename, 'rb') as file:
        metric_data = pickle.load(file)
#     model_name_list = list(metric_data.keys())
    ablation_name_list = list(metric_data[model_name_list[0]].keys())
    ablation_name_list_pretty = [i.split("_",1)[1] for i in ablation_name_list]
    
    df_Corr = pd.DataFrame(columns=model_name_list, index=ablation_name_list)
    df_R2 = pd.DataFrame(columns=model_name_list, index=ablation_name_list)
    
    for key, value in metric_data.items():
        for ab in ablation_name_list:
            try:
                df_Corr.loc[ab, key] = np.round(np.mean(value[ab][:,0]), decimals=3)
                df_R2.loc[ab, key] = np.round(np.mean(value[ab][:,2]), decimals=3)
            except(KeyError):
                df_Corr.loc[ab, key] = 'TBF'
                df_R2.loc[ab, key] = 'TBF'

    # renaming the index
    df_Corr = df_Corr.rename(index=dict(zip(df_Corr.index, ablation_name_list_pretty)))
    df_R2 = df_R2.rename(index=dict(zip(df_R2.index, ablation_name_list_pretty)))
    
    new_columns = [(outcome, col) for col in df_Corr.columns]
    new_column_index = pd.MultiIndex.from_tuples(new_columns)
    
    df_Corr.columns = new_column_index
    df_R2.columns = new_column_index
    if comb_flag==0:
        Corr_comb = df_Corr
        R2_comb = df_R2
    else:
        Corr_comb = pd.concat([Corr_comb, df_Corr], axis=1)
        R2_comb = pd.concat([R2_comb, df_R2], axis=1)
    comb_flag = comb_flag + 1

#     print(" Outcome ", outcome, " Correlation")
#     pprint.pp(df_Corr)
#     print("\n Outcome ", outcome, " R2 score")
#     pprint.pp(df_R2)
pprint.pp(R2_comb)
R2_latex = R2_comb.to_latex()
print(R2_latex)

                                  postop_los               opioids_count_day0  \
                                        XGBT  Scarf TabNet               XGBT   
preops_cbow                            0.207  0.182   -0.0              -0.14   
preops_cbow_homemeds                   0.195  0.176  0.224             -0.127   
preops_cbow_pmh_problist_homemeds      0.206  0.087  0.141             -0.082   

                                                opioids_count_day1         \
                                   Scarf TabNet               XGBT  Scarf   
preops_cbow                       -0.331 -0.211             -6.762 -9.276   
preops_cbow_homemeds              -0.257 -0.268              -6.28  -9.14   
preops_cbow_pmh_problist_homemeds -0.276 -0.287             -5.427  -8.51   

                                           
                                   TabNet  
preops_cbow                       -14.721  
preops_cbow_homemeds              -15.135  
preops_cbow_pmh_problist_homemed

/tmp/ipykernel_2482294/4138674834.py:44: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  R2_latex = R2_comb.to_latex()


In [28]:
# result location 
pred_stage = 'Intraoperative'  # {'Preoperative', 'Intraoperative'}
outcome_list= ['icu', 'mortality', 'aki2']
reg_outcome_list =['postop_los', 'opioids_count_day0', 'opioids_count_day1']
model_name_list = ['XGBTtsSum', 'lstm', 'MVCL']
sav_dir = '/mnt/ris/sandhyat/Output-TS_docker_July2024/Best_results/' + pred_stage +"/"
# sav_dir = '/home/trips/PeriOperative_RiskPrediction/Best_results/' + pred_stage +"/"

In [29]:
comb_flag=0
for outcome in outcome_list:
    perf_filename = sav_dir + str(outcome) + '_Best_perf_metrics_combined_' + pred_stage.lower() + '.pickle'
    with open(perf_filename, 'rb') as file:
        metric_data = pickle.load(file)
#     model_name_list = list(metric_data.keys())
    if 'MVCL' in model_name_list:
        ablation_name_list = list(metric_data['MVCL'].keys())
    else:
        ablation_name_list = list(metric_data[model_name_list[0]].keys())
    ablation_name_list_pretty = [i.split("_",1)[1] for i in ablation_name_list]
    
    df_auroc = pd.DataFrame(columns=model_name_list, index=ablation_name_list)
    df_auprc = pd.DataFrame(columns=model_name_list, index=ablation_name_list)
    for key, value in metric_data.items():
        for ab in ablation_name_list:
            try:
                df_auroc.loc[ab, key] = np.round(np.mean(value[ab][:,0]), decimals=3)
                df_auprc.loc[ab, key] = np.round(np.mean(value[ab][:,1]), decimals=3)
            except(KeyError):
                df_auroc.loc[ab, key] = 'TBF'
                df_auprc.loc[ab, key] = 'TBF'

    # renaming the index
    df_auroc = df_auroc.rename(index=dict(zip(df_auroc.index, ablation_name_list_pretty)))
    df_auprc = df_auprc.rename(index=dict(zip(df_auprc.index, ablation_name_list_pretty)))
    
    new_columns = [(outcome, col) for col in df_auroc.columns]
    new_column_index = pd.MultiIndex.from_tuples(new_columns)
    
    df_auroc.columns = new_column_index
    df_auprc.columns = new_column_index

    if comb_flag==0:
        AUROC_comb = df_auroc
        AUPRC_comb = df_auprc
    else:
        AUROC_comb = pd.concat([AUROC_comb, df_auroc], axis=1)
        AUPRC_comb = pd.concat([AUPRC_comb, df_auprc], axis=1)
    comb_flag = comb_flag + 1
#     print(" Outcome ", outcome, " AUROC")
#     pprint.pp(df_auroc)
#     print("\n Outcome ", outcome, " AUPRC")
#     pprint.pp(df_auprc)
# pprint.pp(AUROC_comb)
# pprint.pp(AUPRC_comb)
print(AUROC_comb)
auroc_latex = AUROC_comb.to_latex()
print(auroc_latex)

                                                         icu                \
                                                   XGBTtsSum   lstm   MVCL   
meds                                                    0.89   0.86  0.847   
flow                                                   0.946  0.914  0.901   
flow_meds                                              0.947  0.907   0.92   
preops_flow_meds                                       0.976  0.965  0.972   
preops_homemeds_flow_meds                              0.978  0.966  0.974   
preops_pmh_problist_homemeds_flow_meds                 0.978   0.96  0.976   
preops_pmh_problist_homemeds_flow_meds_postopco...       TBF    TBF  0.975   

                                                   mortality                \
                                                   XGBTtsSum   lstm   MVCL   
meds                                                   0.883   0.84  0.831   
flow                                                   0.928  0

/tmp/ipykernel_2482294/1340292272.py:48: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  auroc_latex = AUROC_comb.to_latex()


In [32]:
comb_flag=0
for outcome in outcome_list:
    perf_filename = sav_dir + str(outcome) + '_Best_perf_metrics_combined_' + pred_stage.lower() + 'Wave2.pickle'
    with open(perf_filename, 'rb') as file:
        metric_data = pickle.load(file)
#     model_name_list = list(metric_data.keys())
#     if 'MVCL' in model_name_list:
#         ablation_name_list = list(metric_data['MVCL'].keys())
#     else:
    ablation_name_list = list(metric_data[model_name_list[0]].keys())
    ablation_name_list_pretty = [i.split("_",1)[1] for i in ablation_name_list]
    
    df_auroc = pd.DataFrame(columns=model_name_list, index=ablation_name_list)
    df_auprc = pd.DataFrame(columns=model_name_list, index=ablation_name_list)
    for key, value in metric_data.items():
        for ab in ablation_name_list:
            try:
                df_auroc.loc[ab, key] = np.round(np.mean(value[ab][:,0]), decimals=3)
                df_auprc.loc[ab, key] = np.round(np.mean(value[ab][:,1]), decimals=3)
            except(KeyError):
                df_auroc.loc[ab, key] = 'TBF'
                df_auprc.loc[ab, key] = 'TBF'

    # renaming the index
    df_auroc = df_auroc.rename(index=dict(zip(df_auroc.index, ablation_name_list_pretty)))
    df_auprc = df_auprc.rename(index=dict(zip(df_auprc.index, ablation_name_list_pretty)))
    
    new_columns = [(outcome, col) for col in df_auroc.columns]
    new_column_index = pd.MultiIndex.from_tuples(new_columns)
    
    df_auroc.columns = new_column_index
    df_auprc.columns = new_column_index

    if comb_flag==0:
        AUROC_comb = df_auroc
        AUPRC_comb = df_auprc
    else:
        AUROC_comb = pd.concat([AUROC_comb, df_auroc], axis=1)
        AUPRC_comb = pd.concat([AUPRC_comb, df_auprc], axis=1)
    comb_flag = comb_flag + 1
#     print(" Outcome ", outcome, " AUROC")
#     pprint.pp(df_auroc)
#     print("\n Outcome ", outcome, " AUPRC")
#     pprint.pp(df_auprc)
# pprint.pp(AUROC_comb)
# pprint.pp(AUPRC_comb)
print(AUROC_comb)
auroc_latex = AUROC_comb.to_latex()
print(auroc_latex)

                                             icu           mortality       \
                                       XGBTtsSum lstm MVCL XGBTtsSum lstm   
meds                                        0.62  NaN  NaN     0.675  NaN   
flow                                       0.879  NaN  NaN     0.803  NaN   
flow_meds                                  0.879  NaN  NaN     0.741  NaN   
preops_flow_meds                           0.975  NaN  NaN     0.796  NaN   
preops_homemeds_flow_meds                  0.971  NaN  NaN     0.846  NaN   
preops_pmh_problist_homemeds_flow_meds     0.972  NaN  NaN     0.897  NaN   

                                                 aki2            
                                       MVCL XGBTtsSum lstm MVCL  
meds                                    NaN     0.504  NaN  NaN  
flow                                    NaN     0.821  NaN  NaN  
flow_meds                               NaN     0.859  NaN  NaN  
preops_flow_meds                        NaN     0.947

/tmp/ipykernel_2482294/897435755.py:48: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  auroc_latex = AUROC_comb.to_latex()


In [14]:
print(ablation_name_list)

['DataModal_meds', 'DataModal_flow', 'DataModal_flow_meds', 'DataModal_preops_cbow_flow_meds', 'DataModal_preops_cbow_homemeds_flow_meds', 'DataModal_preops_cbow_pmh_problist_homemeds_flow_meds']


In [77]:
comb_flag=0
for outcome in reg_outcome_list:
    perf_filename = sav_dir + str(outcome) + '_Best_perf_metrics_combined_' + pred_stage.lower() + '.pickle'
    with open(perf_filename, 'rb') as file:
        metric_data = pickle.load(file)
    model_name_list = list(metric_data.keys())
    if 'MVCL' in model_name_list:
        ablation_name_list = list(metric_data['MVCL'].keys())
    else:
        ablation_name_list = list(metric_data[model_name_list[0]].keys())
    ablation_name_list_pretty = [i.split("_",1)[1] for i in ablation_name_list]
    
    df_Corr = pd.DataFrame(columns=model_name_list, index=ablation_name_list)
    df_R2 = pd.DataFrame(columns=model_name_list, index=ablation_name_list)
    
    for key, value in metric_data.items():
        for ab in ablation_name_list:
            try:
                df_Corr.loc[ab, key] = np.round(np.mean(value[ab][:,0]), decimals=3)
                df_R2.loc[ab, key] = np.round(np.mean(value[ab][:,2]), decimals=3)
            except(KeyError):
                df_Corr.loc[ab, key] = 'TBF'
                df_R2.loc[ab, key] = 'TBF'

    # renaming the index
    df_Corr = df_Corr.rename(index=dict(zip(df_Corr.index, ablation_name_list_pretty)))
    df_R2 = df_R2.rename(index=dict(zip(df_R2.index, ablation_name_list_pretty)))
    
    new_columns = [(outcome, col) for col in df_Corr.columns]
    new_column_index = pd.MultiIndex.from_tuples(new_columns)
    
    df_Corr.columns = new_column_index
    df_R2.columns = new_column_index

    if comb_flag==0:
        Corr_comb = df_Corr
        R2_comb = df_R2
    else:
        Corr_comb = pd.concat([Corr_comb, df_Corr], axis=1)
        R2_comb = pd.concat([R2_comb, df_R2], axis=1)
    comb_flag = comb_flag + 1

#     print(" Outcome ", outcome, " Correlation")
#     pprint.pp(df_Corr)
#     print("\n Outcome ", outcome, " R2 score")
#     pprint.pp(df_R2)
pprint.pp(R2_comb)
R2_latex = R2_comb.to_latex()
print(R2_latex)

                                       postop_los        opioids_count_day0  \
                                        XGBTtsSum   lstm          XGBTtsSum   
meds                                        0.251  0.213              0.241   
flow                                        0.336    TBF              0.268   
flow_meds                                   0.359    TBF              0.283   
preops_flow_meds                            0.412    TBF              0.337   
preops_homemeds_flow_meds                   0.432    TBF              0.345   
preops_pmh_problist_homemeds_flow_meds      0.571    TBF              0.338   

                                              opioids_count_day1         
                                         lstm          XGBTtsSum   lstm  
meds                                    0.164              0.312  0.262  
flow                                      TBF              0.354    TBF  
flow_meds                                 TBF              0.376    TBF

/tmp/ipykernel_1239913/3034832231.py:48: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  R2_latex = R2_comb.to_latex()


In [34]:
comb_flag=0
for outcome in reg_outcome_list:
    perf_filename = sav_dir + str(outcome) + '_Best_perf_metrics_combined_' + pred_stage.lower() + 'Wave2.pickle'
    with open(perf_filename, 'rb') as file:
        metric_data = pickle.load(file)
#     model_name_list = list(metric_data.keys())
#     if 'MVCL' in model_name_list:
#         ablation_name_list = list(metric_data['MVCL'].keys())
#     else:
    ablation_name_list = list(metric_data[model_name_list[0]].keys())
    ablation_name_list_pretty = [i.split("_",1)[1] for i in ablation_name_list]
    
    df_Corr = pd.DataFrame(columns=model_name_list, index=ablation_name_list)
    df_R2 = pd.DataFrame(columns=model_name_list, index=ablation_name_list)
    
    for key, value in metric_data.items():
        for ab in ablation_name_list:
            try:
                df_Corr.loc[ab, key] = np.round(np.mean(value[ab][:,0]), decimals=3)
                df_R2.loc[ab, key] = np.round(np.mean(value[ab][:,2]), decimals=3)
            except(KeyError):
                df_Corr.loc[ab, key] = 'TBF'
                df_R2.loc[ab, key] = 'TBF'

    # renaming the index
    df_Corr = df_Corr.rename(index=dict(zip(df_Corr.index, ablation_name_list_pretty)))
    df_R2 = df_R2.rename(index=dict(zip(df_R2.index, ablation_name_list_pretty)))
    
    new_columns = [(outcome, col) for col in df_Corr.columns]
    new_column_index = pd.MultiIndex.from_tuples(new_columns)
    
    df_Corr.columns = new_column_index
    df_R2.columns = new_column_index

    if comb_flag==0:
        Corr_comb = df_Corr
        R2_comb = df_R2
    else:
        Corr_comb = pd.concat([Corr_comb, df_Corr], axis=1)
        R2_comb = pd.concat([R2_comb, df_R2], axis=1)
    comb_flag = comb_flag + 1

#     print(" Outcome ", outcome, " Correlation")
#     pprint.pp(df_Corr)
#     print("\n Outcome ", outcome, " R2 score")
#     pprint.pp(df_R2)
pprint.pp(R2_comb)
R2_latex = R2_comb.to_latex()
print(R2_latex)

                                       postop_los            \
                                        XGBTtsSum lstm MVCL   
meds                                       -0.142  NaN  NaN   
flow                                       -0.426  NaN  NaN   
flow_meds                                   0.094  NaN  NaN   
preops_flow_meds                            0.004  NaN  NaN   
preops_homemeds_flow_meds                   0.039  NaN  NaN   
preops_pmh_problist_homemeds_flow_meds      0.067  NaN  NaN   

                                       opioids_count_day0            \
                                                XGBTtsSum lstm MVCL   
meds                                               -0.154  NaN  NaN   
flow                                               -0.319  NaN  NaN   
flow_meds                                          -0.234  NaN  NaN   
preops_flow_meds                                    -0.28  NaN  NaN   
preops_homemeds_flow_meds                          -0.433  NaN  NaN  

/tmp/ipykernel_2482294/191078315.py:48: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  R2_latex = R2_comb.to_latex()
